# Issue 132 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Output 
* test 1


In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()
print(timestamp)  # Outputs seconds since Unix epoch


start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


1753715535.951369
Start: 2025-07-28 17:12:15


In [2]:
# Toaletter inom 200 meter fran Stockholm Archipelago Trail (via Wikidata P402)

# Installera beroenden om de saknas
try:
    import geopandas as gpd
    import shapely
    import folium
    import requests
    from SPARQLWrapper import SPARQLWrapper, JSON
except ImportError:
    import sys
    !{sys.executable} -m pip install geopandas shapely folium requests SPARQLWrapper

import requests
import geopandas as gpd
from shapely.geometry import LineString, Point, MultiLineString
from shapely.ops import unary_union
import folium
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
from datetime import datetime
from folium import FeatureGroup, LayerControl, Marker, Icon

# Steg 1: Hämta OSM relationer via Wikidata (Stockholm Archipelago Trail = Q131318799)
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid ?wikipedia ?image ?length ?official
WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  OPTIONAL { ?item wdt:P18 ?image. }
  OPTIONAL { ?item wdt:P2043 ?length. }
  OPTIONAL {
    ?item p:P856 ?officialClaim.
    ?officialClaim ps:P856 ?official.
    ?officialClaim pq:P407 wd:Q9027.
  }
  OPTIONAL { ?wikipedia schema:about ?item; schema:isPartOf <https://sv.wikipedia.org/>. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for result in results['results']['bindings']:
    etapper.append({
        "id": result['osmid']['value'],
        "label": result.get('itemLabel', {}).get('value', ''),
        "island": result.get('islandLabel', {}).get('value', ''),
        "wikipedia": result.get('wikipedia', {}).get('value', ''),
        "image": result.get('image', {}).get('value', ''),
        "length": result.get('length', {}).get('value', ''),
        "official": result.get('official', {}).get('value', '')
    })
osm_ids = [e['id'] for e in etapper]
print(f"Hämtade {len(osm_ids)} OSM-relationer från Wikidata")

# Steg 2: Hämta geometrier från Overpass
features = []
color_palette = [
    "blue", "green", "purple", "orange", "darkred", "cadetblue",
    "lightgray", "darkblue", "darkgreen", "pink", "lightblue",
    "lightgreen", "gray", "black", "beige", "lightred"
]
overpass_url = "http://overpass-api.de/api/interpreter"
for rel_id in osm_ids:
    overpass_query = f"""
    [out:json];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    response = requests.post(overpass_url, data={'data': overpass_query})
    if response.status_code != 200:
        print(f"Fel vid hämtning av relation {rel_id}: {response.text}")
        continue
    data = response.json()
    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lon'], pt['lat']) for pt in element['geometry']]
            features.append(LineString(coords))

if not features:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Gruppera geometrier per relation
geom_per_rel = defaultdict(list)
for e, geom in zip(etapper, features):
    geom_per_rel[e['id']].append(geom)

etapp_geoms = []
for e in etapper:
    geoms = geom_per_rel[e['id']]
    etapp_geoms.append(MultiLineString(geoms) if len(geoms) > 1 else geoms[0])

# Visualisering
trail_layers = []
for i, (meta, geom) in enumerate(zip(etapper, etapp_geoms)):
    color = color_palette[i % len(color_palette)]
    popup = f"<b>{meta['label']}</b><br>Ö: {meta['island']}<br>Längd: {meta['length']} km"
    if meta['image']:
        popup += f"<br><img src='{meta['image']}' width='200'>"
    if meta['official']:
        popup += f"<br><a href='{meta['official']}' target='_blank'>Officiell länk</a>"
    if meta['wikipedia']:
        popup += f"<br><a href='{meta['wikipedia']}' target='_blank'>Wikipedia ({meta['island']})</a>"
    popup += f"<br><a href='https://www.openstreetmap.org/relation/{meta['id']}' target='_blank'>OSM-relation</a>"

    trail_layers.append(folium.GeoJson(geom, name=meta['label'],
                                       style_function=lambda x, color=color: {"color": color},
                                       tooltip=meta['label'], popup=popup))

gdf_trail = gpd.GeoDataFrame(geometry=features, crs="EPSG:4326")
meta_gdf = gpd.GeoDataFrame(etapper, geometry=etapp_geoms, crs="EPSG:4326")
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
meta_gdf.to_file(f"../kartor/sat_etapper_{timestamp}.geojson", driver="GeoJSON")
meta_gdf.drop(columns="geometry").to_csv(f"../kartor/sat_etapper_{timestamp}.csv", index=False)

# Buffra 200 meter
trail_utm = gdf_trail.to_crs(epsg=3006)
buffered_utm = trail_utm.buffer(200)
buffered = buffered_utm.to_crs(epsg=4326)

# Toaletter från OSM
bbox = gdf_trail.total_bounds
query_toilets = f"""
[out:json][timeout:25];
node["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out;
"""
toilets_response = requests.post(overpass_url, data={'data': query_toilets})
toilets_data = toilets_response.json()

toilet_points = []
for el in toilets_data['elements']:
    if el['type'] == 'node':
        pt = Point(el['lon'], el['lat'])
        tags = el.get('tags', {})
        toilet_points.append({"geometry": pt, "tags": tags, "id": el['id']})

gdf_toilets = gpd.GeoDataFrame(toilet_points, crs="EPSG:4326")
in_range = gdf_toilets[gdf_toilets.geometry.within(buffered.geometry.unary_union)]

in_range.to_file(f"../kartor/sat_toaletter_{timestamp}.geojson", driver="GeoJSON")
in_range.drop(columns="geometry").to_csv(f"../kartor/sat_toaletter_{timestamp}.csv", index=False)

# Folium-karta
m = folium.Map(location=[gdf_trail.geometry[0].centroid.y, gdf_trail.geometry[0].centroid.x], zoom_start=10)

for i, layer in enumerate(trail_layers):
    layer.add_to(m)
    if hasattr(etapp_geoms[i], 'geoms'):
        start_point = etapp_geoms[i].geoms[0].coords[0]
    else:
        start_point = etapp_geoms[i].coords[0]
    Marker(
        location=(start_point[1], start_point[0]),
        icon=Icon(color='green', icon='play', prefix='fa'),
        tooltip=etapper[i]['label']
    ).add_to(m)

folium.GeoJson(buffered.geometry[0], name="200m Buffert",
               style_function=lambda x: {
                   'fillColor': '#0000ff', 'color': '#0000ff',
                   'weight': 1, 'fillOpacity': 0.1
               }).add_to(m)

good_group = FeatureGroup(name="Godkända toaletter")
warn_group = FeatureGroup(name="Varningar")

for idx, row in in_range.iterrows():
    tags = row['tags']
    el_id = row['id']
    osm_url = f"https://www.openstreetmap.org/node/{el_id}"

    required_tags = ["amenity", "access", "unisex", "toilets:disposal", "wheelchair"]
    recommended_tags = ["toilets:handwashing", "toilets:paper_supply", "opening_hours", "fee", "operator", "description"]
    problem_tags = {"access": "private", "key": "yes", "fee": "yes"}

    missing = [t for t in required_tags if t not in tags]
    warnings = [f"⚠️ {k}={v} bör undvikas" for k, v in problem_tags.items() if tags.get(k) == v]
    recommended_missing = [f"➕ {t} saknas" for t in recommended_tags if t not in tags]

    status_summary = "✅ Minimikrav uppfyllda" if not missing else f"⚠️ Saknar: {', '.join(missing)}"
    suggestions = status_summary + "<br>" + "<br>".join(warnings + recommended_missing + ["<i>Tips: kontrollera viktiga attribut</i>"])

    popup_text = f"<b>OSM objekt</b><br><a href='{osm_url}' target='_blank'>{osm_url}</a><br><br>{suggestions}"
    icon_type = "info-sign"
    icon_color = "red" if warnings or missing else "blue"

    Marker(location=[row.geometry.y, row.geometry.x],
           popup=popup_text,
           icon=Icon(icon=icon_type, color=icon_color)).add_to(warn_group if warnings or missing else good_group)

m.add_child(good_group)
m.add_child(warn_group)
LayerControl().add_to(m)
output_path = f"../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
m.save(output_path)
print(f"Karta sparad: {output_path}")

Hämtade 20 OSM-relationer från Wikidata
Karta sparad: ../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html


/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_12167/3855737809.py:145: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  in_range = gdf_toilets[gdf_toilets.geometry.within(buffered.geometry.unary_union)]


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-07-28 17:12:50
Total time elapsed: 34.98 seconds
